# Dataset: *germanwings*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [10]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-src.csv', encoding = 'utf-8', header =0)
print(df.shape)
print(src_tw_df.shape)

(4020, 11)
(405, 7)


In [11]:
df.isnull().sum()

src_tweet_id          0
src_user_id           0
src_tweet             0
src_date              0
reply_tweet_id        0
reply_user_id         0
reply_tweet           0
reply_date            0
label                 0
cleaned_reply_tw    416
reply_sentiment       0
dtype: int64

In [12]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)

In [5]:
src_tw_df.isnull().sum()

src_tweet_id      0
src_user_id       0
src_tweet         0
src_date          0
label             0
cleaned_src_tw    1
src_sentiment     0
dtype: int64

In [6]:
src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet

390    RT @khjelmgaard: German media reporting #Andre...
Name: src_tweet, dtype: object

In [9]:
a = src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet_id.values
df.loc[df.src_tweet_id.isin(a),['reply_tweet','cleaned_reply_tw']]

,reply_tweet,cleaned_reply_tw
3766,@usatoday @khjelmgaard davvero?! Effettivament...,davvero effettivamente viviamo un mondo fantas...
3767,"@USATODAY Col @OliverNorthFNC, military like y...",col military like gens j keane say obama bomb ...
3768,@USATODAY @khjelmgaard Him and millions of oth...,millions others no doubt tip six years later c...
3769,@USATODAY best follow I ever did,best follow ever
3770,@USATODAY @khjelmgaard #AndreasLubitz #GermanW...,andreaslubitz germanwings dred splitup girlfri...
3771,@USATODAY @khjelmgaard \n #A320 #PilotsNoFlyNo...,pilotsnoflynosedoors offcourse barcelonadussel...
3772,@EdWardMDBlog @USATODAY @khjelmgaard all rumor...,rumorsnot confirm
3773,@tinkalee_12 @USATODAY @khjelmgaard My 2nd sou...,nd source confirm french media report fer unve...
3774,@tinkalee_12 @USATODAY @khjelmgaard #F4Phantom...,fphantom crash mph hit concretewall dred
3775,@USATODAY @khjelmgaard ers a link to prove @An...,ers link prove intentionally set kill


## Emotion Analysis

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [13]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [14]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [15]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [16]:
df['reply_emo_ro'] = emotion_ro

In [17]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-df-2.csv',index=False)

In [18]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [19]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [20]:
print("source tweet:", df.src_tweet[1118],'\n')
print("original reply tw: ", df.reply_tweet[1118])
print("cleaned reply tw: ", df.cleaned_reply_tw[1118])

print('roBERT based: ',df.reply_emo_ro[1118])


source tweet: Crash of the Germanwings flight is extremely unfortunate. We share the sorrow &amp; pain of the families of those on board. 

original reply tw:  @narendramodi  really its bad news
cleaned reply tw:  really bad news
roBERT based:  sadness
